### Demand forecasting For FMCG Company

In [ ]:
!unzip "/content/mate_labs_demand_forecasting_hiring_challenge-dataset.zip"

Archive:  /content/mate_labs_demand_forecasting_hiring_challenge-dataset.zip
replace Data/Train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Data/Train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Data/Train.csv          
  inflating: Data/Test.csv           
  inflating: Data/Sample Submission.csv  
  inflating: Data/Promotional_Data.csv  
  inflating: Data/.~lock.Sample Submission.csv#  


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from fbprophet import Prophet
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

Reading the csv file and storing it as a dataframe

In [ ]:
train = pd.read_csv("/content/Data/Train.csv")
test = pd.read_csv("/content/Data/Test.csv")
promo = pd.read_csv("/content/Data/Promotional_Data.csv")

Converting to the datetime format using Pandas

In [ ]:
train['DATE'] = pd.to_datetime(train['DATE'])
test['DATE'] = pd.to_datetime(test['DATE'])
promo['DATE'] = pd.to_datetime(promo['DATE'])

# Promo csv Preprocessing

In [ ]:
promo['DB Channel'] = promo['DB Channel'].replace(['All'],'0,1,2')
promo['DB Channel'] = promo['DB Channel'].replace(['2 only'],'2')
promo['DB Channel'] = promo['DB Channel'].replace(['1&0 only'],'0,1')

In [ ]:
def splitDataFrameList(df,target_column,separator):
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [ ]:
promof = splitDataFrameList(promo,'DB Channel', ',')

In [ ]:
promof.rename(columns={'DB Channel':'Channel'}, inplace=True)

In [ ]:
promof['Channel'] = promof['Channel'].astype(str).astype(int) 

In [ ]:
def describe_data(df):
    print("Data Types:")
    print(df.dtypes)
    print("Rows and Columns:")
    print(df.shape)
    print("Column Names:")
    print(df.columns)
    print("Null Values:")
    print(df.apply(lambda x: sum(x.isnull()) / len(df)))

# Train csv Preprocessing

In [ ]:
train.rename(columns={'Secondary Sales Amount STC':'SSAS','Secondary Sales Amount Invoiced':'SSAI','Secondary Sales Amount Returned':'SSAR','Secondary Sales Quantity Invoiced':'SSQI','Secondary Sales Quantity Returned':'SSQR'}, inplace=True)
train.drop(['SSAS','SSAI','SSAR','SSQR'], 1, inplace=True)

In [ ]:
trainf = pd.merge(train, promof, how='left', on=["DATE",'SKU Code','Channel']) 

In [ ]:
trainf["Promo"] = trainf["Promo"].fillna(0)

In [ ]:
trainf["year"] = pd.to_datetime(trainf["DATE"]).dt.year
trainf["month"] = pd.to_datetime(trainf["DATE"]).dt.month
trainf["day"] = pd.to_datetime(trainf["DATE"]).dt.day
trainf.drop(["DATE"], 1, inplace=True)

# Test csv Preprocessing

In [ ]:
testf = test.copy()

In [ ]:
describe_data(testf)

Data Types:
SKU Code                    int64
CFA Code                    int64
Channel                     int64
Quantity                    int64
DATE               datetime64[ns]
SKU Combination             int64
dtype: object
Rows and Columns:
(2609, 6)
Column Names:
Index(['SKU Code', 'CFA Code', 'Channel', 'Quantity', 'DATE',
       'SKU Combination'],
      dtype='object')
Null Values:
SKU Code           0.0
CFA Code           0.0
Channel            0.0
Quantity           0.0
DATE               0.0
SKU Combination    0.0
dtype: float64


In [ ]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    diff_df.to_csv('diff.csv',index = False)
    return diff_df

In [ ]:
testf = pd.merge(test, promof, how='left', on=["DATE",'SKU Code','Channel']) 

In [ ]:
testf["Promo"] = testf["Promo"].fillna(0)

In [ ]:
test["year"] = pd.to_datetime(test["DATE"]).dt.year
test["month"] = pd.to_datetime(test["DATE"]).dt.month
test["day"] = pd.to_datetime(test["DATE"]).dt.day
test.drop(["DATE"], 1, inplace=True)

# Model Training

In [ ]:
from sklearn.model_selection import train_test_split
X = trainf.drop('SSQI', axis=1)
y = trainf['SSQI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor(random_state=0)
model = regressor.fit(X_train,y_train)

In [ ]:
from sklearn import linear_model
reg = linear_model.BayesianRidge()
model = reg.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(random_state=1, n_estimators=10)
model = reg.fit(X_train,y_train)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
model = ereg.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(testf)

In [ ]:
y_pred

In [ ]:
output = pd.read_csv("/content/Data/Test.csv")

In [ ]:
testf["Secondary Sales Quantity Invoiced"] = y_pred

In [ ]:
testf['DATE'] = pd.to_datetime(testf.year*10000+testf.month*100+testf.day,format='%Y%m%d')
testf.drop(['year','month','day'],1,inplace=True)

In [ ]:
testf.drop(['SKU Combination','CFA Code','Quantity'],1,inplace=True)

In [ ]:
testf.drop(['Promo'],1,inplace=True)

In [ ]:
testf.to_csv('testf.csv', index=False)